In [8]:
#!pip install cufflinks
#!pip install textblob 
#!pip install wordcloud 

In [10]:
import numpy as np
import pandas as pd
import nltk
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import cufflinks as cf
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
from wordcloud import WordCloud
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = True)
cf.go_offline();
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")
pd.set_option("display.max_columns", None)

In [14]:
df = pd.read_csv("amazon.csv")
df.sort_values("wilson_lower_bound", ascending=False, inplace=True)
df.drop("Unnamed: 0", inplace=True, axis=1)

### Missing Values & Plotting

In [17]:
def missing_values_analysis(df):
  na_columns_ = [col for col in df.columns if df[col].isnull().sum() > 0] # ['reviewerName', 'reviewText']
  n_miss = df[na_columns_].isnull().sum().sort_values(ascending=True)
  ratio_ = (df[na_columns_].isnull().sum() / df.shape[0]*100)
  missing_df = pd.concat([n_miss, np.round(ratio_, 2)], axis=1, keys=["Missing Values", "Ratio"])
  #missing_df = pd.DataFrame(missing_df)
  return missing_df

def check_dataframe(df, head=5, tail=5):
  print("SHAPE".center(52, '~'))
  print(f"Rows: {df.shape[0]}")
  print(f"Columns: {df.shape[1]}")
  print("TYPES".center(52, '~'))
  print(df.dtypes)
  print("".center(52, '~'))
  print(missing_values_analysis(df))
  print("DUPLICATED VALUES".center(52, '~'))
  print(df.duplicated().sum())
  print("QUANTITILES".center(52, '~'))
  print(df.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

def check_uniqueness(df):
  nunique_df = pd.DataFrame({'Variable': df.columns,
                             'Classes': [df[i].nunique() for i in df.columns]})
  nunique_df.sort_values('Classes', ascending=False, inplace=True)
  nunique_df.reset_index(drop=True, inplace=True)
  return nunique_df

check_dataframe(df)
print("UNIQUENESS".center(52, '~'))
print(check_uniqueness(df))

~~~~~~~~~~~~~~~~~~~~~~~SHAPE~~~~~~~~~~~~~~~~~~~~~~~~
Rows: 4915
Columns: 11
~~~~~~~~~~~~~~~~~~~~~~~TYPES~~~~~~~~~~~~~~~~~~~~~~~~
reviewerName             object
overall                   int64
reviewText               object
reviewTime               object
day_diff                  int64
helpful_yes               int64
helpful_no                int64
total_vote                int64
score_pos_neg_diff        int64
score_average_rating    float64
wilson_lower_bound      float64
dtype: object
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
              Missing Values  Ratio
reviewerName               1   0.02
reviewText                 1   0.02
~~~~~~~~~~~~~~~~~DUPLICATED VALUES~~~~~~~~~~~~~~~~~~
0
~~~~~~~~~~~~~~~~~~~~QUANTITILES~~~~~~~~~~~~~~~~~~~~~
                       0.00  0.05   0.50        0.95       0.99         1.00
overall                 1.0   2.0    5.0    5.000000    5.00000     5.000000
day_diff                1.0  98.0  431.0  748.000000  943.00000  1064.000000
helpf

### Categorical Variable Analysis

In [20]:
constraints = ["#B34D22", "#EBE00C", "#1FEB0C", "#0C92EB", "#EB0CD5"]
def categorical_variable_summary(df, column_name):
  fig = make_subplots(rows = 1, cols=2, 
                      subplot_titles=('Countplot', 'Percentage'), 
                      specs=[[{'type':"xy"},{'type':"domain"}]])
  
  fig.add_trace(go.Bar(y = df[column_name].value_counts().values.tolist(),
                       x = [str(i) for i in df[column_name].value_counts().index],
                       text = df[column_name].value_counts().values.tolist(),
                       textfont = dict(size=14),
                       name = column_name,
                       textposition = 'auto',
                       showlegend = False,
                       marker = dict(color=constraints,
                                     line=dict(color='#DBE6EC', width=1))),
                row = 1, col = 1)
  
  fig.add_trace(go.Pie(labels=df[column_name].value_counts().keys(),
                       values=df[column_name].value_counts().values,
                       textfont = dict(size=18),
                       name = column_name,
                       textposition = 'auto',
                       showlegend = False,
                       marker = dict(colors=constraints)),
                row = 1, col = 2)
  
  fig.update_layout(title={'text': column_name, 
                           'y':0.9, 'x':0.5, 
                           'xanchor':"center", 'yanchor':'top'},
                           template='plotly_white')
  iplot(fig)
    
    
categorical_variable_summary(df, "overall")

In [21]:
df.head(3)

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,I have tested dozens of SDHC and micro-SDHC ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,NOTE: please read the last update (scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139


###### Credits: https://www.youtube.com/watch?v=SMecj9AIDF8